# Conduite cylindrique : comparaison des méthodes numériques


In [ ]:
from trustutils import run 
from trustutils.jupyter import plot
import numpy as np
from functions import *
from math import *
import matplotlib.pyplot as plt

run.introduction("M. El Moatamid","08/06/2023")

run.TRUST_parameters()

### Description 

On simule un écoulement monophasique turbulent en RANS dans une conduite circulaire (tube) avec les différentes configurations: 
- VDF, VEF et PolyMAC (schémas numériques)
- k-epsilon, k-omega et k-tau (modèles de turbulence)

La simulation est faite sur un tube de longueur L=100m et rayon R=2m

Les maillages utilisées avec VEF et Polymac sont générés avec salomé (voir script python dans src/)

 Pour VDF, on utilise un maillages 2D avec une condition limite d'axisymétrie en x=0
    
                                            paroi
                _______________________________________________________________
                |                                                             |
    Inlet:      |                                                             | Outlet:
      velocity->|                                                             |-> pressure
      k       ->|                                                             |-> k
      epsilon ->|                                                             |-> epsilon
                |                                                             |
                |_____________________________________________________________|
                                         axisymétrie


## Préparation des calculs
Attention : 
L'axe normal à la paroi est différent selon les schémas et maillages utilisés (x pour VDF et r pour les maillages 3D). Nous allons noter l'axe normal à la paroi "y" pour toutes les méthodes de calculs utilisée (idem pour l'axe de la paroi qu'on note "x" dans toute la fiche).

In [ ]:
# # Configuration

# Liste des configuration disponible : ["VEF_k-epsilon", "VDF_k-epsilon", "VDF_k-tau", "VDF_k-omega", "PolyMAC_k-tau", "PolyMAC_k-omega"]
config = ["VEF_k-epsilon", "VDF_k-epsilon", "VDF_k-tau", "VDF_k-omega", "PolyMAC_k-tau", "PolyMAC_k-omega"]
config = ["VDF_k-epsilon", "VDF_k-tau", "VDF_k-omega", "PolyMAC_k-tau", "PolyMAC_k-omega"]
# Il est possible de : changer l'ordre de la liste config + mettre uniquement une partie des config disponibles
# Pour lancer un cas VEF k-epsilon, vous devez generer le maillage sur votre machine en en lancant le fichier maillage_tube_cut_vef.py avec Salome 



# # Parallelisation du calcul

number_of_partitions = 2

# # Fluide

mu = 0.02
rho = 1000

# # Conditions limites

inlet_velocity = 1
outlet_pressure = 1e5
inlet_k = 0.01
inlet_epsilon = 0.005

# # Maillage : nombre de noeuds

# nombre de noeuds selon le rayon du cylindre : VDF axi
#Ny = [21,25] 
#N_vef = [11,13]
Ny = [21,25]
N_vef = [11,13]

# # Boucle temporelle

tmax = 300 # [s]

# # Post traitement

# plot profiles
x_prof = 95 # position selon x pour extraire différents profils (vitesse, k, ...)
y_min_prof = sonde_firstpoint(2,Ny)
y_min_profvef = sonde_firstpoint(2,N_vef)
y_max_prof = 2-sonde_firstpoint(2,Ny)
y_max_profvef = 2-sonde_firstpoint(2,N_vef)
nb_points_prof = [N-1 for N in Ny]


### Premier calculs sur les paramètres entrés
On calcule sur la base des paramètres de simulation choisis par l'utilisateur :
- Le nombre de Reynolds
- Une estimaction du coefficient de frottement Cf qui permet d'estimer (avec une bonne approximation) tau_w, utau et y⁺

L'utilisateur peut donc savoir à quels résultats s'attendre et vérifier que les valeurs de y+ permettent d'utiliser une loi de paroi standard (VEF)

In [ ]:
R = 2 # rayon conduit [m]
Dh = 4*(pi*R**2)/(2*pi*R) # diamètre hydraulique
Re_Dh = Reynolds(inlet_velocity, rho, Dh, mu) # calcul du reynolds
cf_pred = cf_prediction(kappa, B, Re_Dh/4, cf_guess) # estimation du coefficient de frottement
tau_pred, u_tau_pred = tau_and_u_tau(cf_pred,1,1000)
print(f"Reynolds : Re_Dh = {Re_Dh:.1e}\n\nEstimations :\n\tcf = {cf_pred:.4f} \ttau_w = {tau_pred:.2f} Pa\tu_tau = {u_tau_pred:.3f} m/s")
print("\nMaillages :\n Valeurs estimées de y+ à la paroi :\n")
y_wall_list = {} # hauteur maille à la paroi
y_plus_list = {} # y+ paroi
y_wall_list["VEF"] = y_wall_list["VDF"] = y_wall_list["PolyMAC"] = [0 for x in range(len(Ny))]
y_plus_list["VEF"] = y_plus_list["VDF"] = y_plus_list["PolyMAC"] = [0 for x in range(len(Ny))]
for i in range(len(Ny)):
    y_VDF, y_VEF, y_PolyMAC = y_wall(R,Ny[i]) # valeurs de y à l'endroit où la vitesse à la paroi est calculée
    _, y_VEF, _ = y_wall(R,N_vef[i])
    y_wall_list["VEF"][i], y_wall_list["VDF"][i], y_wall_list["PolyMAC"][i] = y_VEF, y_VDF, y_PolyMAC
    y_VDF_plus, y_VEF_plus, y_PolyMAC_plus = y_plus_prediction(R,Ny[i],u_tau_pred,mu,rho)
    _, y_VEF_plus, _ = y_plus_prediction(R,N_vef[i],u_tau_pred,mu,rho)
    y_plus_list["VEF"][i], y_plus_list["VDF"][i], y_plus_list["PolyMAC"][i] = y_VEF_plus, y_VDF_plus, y_PolyMAC_plus
    print(f" - mesh_{i} : y+_VDF = {y_VDF_plus:.1f}\t y+_VEF = {y_VEF_plus:.1f}\t y+_PolyMAC = {y_PolyMAC_plus:.1f}")

## Initialisation et Lancement des calculs

In [ ]:
force_recalculation = True # put True to force recalculation
build = run.BUILD_DIRECTORY # build directory (absolute path !)
config_Pb_hydr = [config[i] for i in range(len(config)) if config[i] in available_config_pb_hydr]
config_Pb_multi = [config[i] for i in range(len(config)) if config[i] in available_config_pb_multi]

In [ ]:
nb_mesh = len(Ny)
nb_method = len(config)
# paramètres utilisés pour la substitution : voir function.py
params = [config, Ny, N_vef, inlet_velocity,outlet_pressure,inlet_k,inlet_epsilon,x_prof,mu,rho,y_min_prof,y_max_prof,y_min_profvef,y_max_profvef,nb_points_prof,tmax]
# # Substitution dictionnary
dict_list = substitution(params)
# data file names  : nous avons 4 jdd de base qui seront modifiés pour lancer les différentes config
names = file_name(config)

import os   
if force_recalculation or not os.path.exists(build):
    run.reset() # Delete the build directory and empty the list of cases to be executed
    run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)
    for i in range(nb_method):
        file = names[i]
        os.system(f"mkdir {build}/{config[i]}")
        for j in range(nb_mesh):
            os.system(f"mkdir {build}/{config[i]}/mesh_{j}")
            if number_of_partitions == 1:
                run.addCaseFromTemplate(f"{file}.data",directory=build+f"/{config[i]}/mesh_{j}",d=dict_list[i][j])
            else:
                os.system(f"cp {build}/*.med {build}/{config[i]}/mesh_{j}")
                GenerateInputFile(f"{build}/{config[i]}/mesh_{j}",build,dict_list[i][j], file)
                os.system(f'cd {build}/{config[i]}/mesh_{j};  make_PAR.data {file} {number_of_partitions}; cd ../../..')
                run.addCase(f"{config[i]}/mesh_{j}", f"PAR_{file}.data",nbProcs=number_of_partitions)
    run.printCases()
    run.runCases() # lance les différents cas test écrits dans build/
    perf = run.tablePerf() # tableau des performances de calcul
else:
    print("Using previous calculation")
    
if number_of_partitions!=1:
    names = ["PAR_"+name for name in names]

### Statistiques

In [ ]:
# Tableau des performances de calcul
import pandas as pd

performance = open(build+"/perf.csv", "w")
for i in range(nb_method):
    file = names[i]
    for j in range(nb_mesh):
        performance.write(read_perf(build+f"/{config[i]}/mesh_{j}/{file}.perf", file, config[i]+f" mesh_{j}"))
performance.close() 
performance = open(build+"/perf.csv", "r")
data = pd.read_csv(performance, sep=" ", header=None)
df = data[data.columns[:-1]]
df.columns = ["Configuration", "Mesh", "Host", "System", "total_CPU_time", "CPU time/step", "nb_cells"]
blankIndex=[''] * len(df)
df.index=blankIndex
display(df)

In [ ]:
grpd = df.groupby('Configuration')
for name, data in grpd:
    plt.plot(data.nb_cells.values, data.total_CPU_time.values, 'o-', label = name)
plt.legend()
plt.xlabel("number of cells")
plt.xscale('log')
plt.ylabel("Total CPU time [s]")
plt.title("Calculation time")
plt.grid()

### Evolution des résidus

In [ ]:
import matplotlib.pyplot as plt
nb_col = max(int(nb_method/2),2)
fig = plt.figure(figsize=(15,ceil(nb_method/nb_col)*4))
axs = fig.subplots(ceil(nb_method/nb_col),nb_col)

for i in range(nb_method):
    file = names[i]
    for j in range(nb_mesh):
        res_file = build+f"/{config[i]}/mesh_{j}/{file}.dt_ev"
        max_residu = plot.loadText(res_file)[3,:]
        # find what is the max residual (called res_name)
        res_name = ""
        for k, res in enumerate(plot.loadText(res_file)[4:,-1]):
            if max_residu[-1] == res:
                res_name = get_residu_name(res_file,k)
        temps = plot.loadText(res_file)[0,:]
        axs[int(i/nb_col),i%nb_col].plot(temps, max_residu,label=f"mesh_{j}")
        axs[int(i/nb_col),i%nb_col].text(0.35, 0.3-j*0.1, f'mesh_{j} : {res_name}', horizontalalignment='center', verticalalignment='center', transform=axs[int(i/nb_col),i%nb_col].transAxes)
    axs[int(i/nb_col),i%nb_col].legend(fontsize=10)
    axs[int(i/nb_col),i%nb_col].set_xlabel("time", fontsize=10)
    axs[int(i/nb_col),i%nb_col].set_ylabel("maxResidu", fontsize=10)
    axs[int(i/nb_col),i%nb_col].set_yscale('log')
    axs[int(i/nb_col),i%nb_col].set_title(f'{config[i]}', fontsize=12)
fig.suptitle(f"Residuals")
fig.tight_layout()

### Maillage
décommenter pour voir le maillage

In [ ]:
#from trustutils import visit
#for i in range(nb_method):
#    visit.showMesh(build+f"/{config[i]}/mesh_0/{names[i]}.lata","dom")

## Post-traitement des résultats
### Contraintes pariétales
Vérifier que la couche limite est bien développée : tracé de tau_wall(x)

In [ ]:
# # Calcul de u_tau (noté u*) pour les différents maillages
# # tracé de tau_w le long de la paroi : vérifier que la couche limite est bien developpée
print(f"Calcul des valeurs de u_tau à x={x_prof}m ainsi que y+ à la fontière sup de la 1ère maille pour VDF et PolyMAC et à la hauteur des centres des faces pour VEF :\n\n")
u_star_list = [[0 for x in range(nb_mesh)] for y in range(nb_method)] # list des u_tau calculés à x_prof
y_plus_cal_list = [[0 for x in range(nb_mesh)] for y in range(nb_method)] # list des y+ calculés à x_prof

# valeur min et max de tau (sert à mettre toutes les courbes à la meme échelle)
min_tau = 50
max_tau = 0

fig = plt.figure(figsize = (15,4*ceil(nb_method/2)))
axs = fig.subplots(ceil(nb_method/2),2)
for i in range(nb_method):
    for j in range(nb_mesh):
        if config[i] in config_Pb_hydr: # avec pb_hydraulique : tau_w déduit des données dans le fichier Ustar.face
            #if "VEF" in config[i]:
            # plot tau_x=f(x)
            wall_file = build+f'/{config[i]}/mesh_{j}/{names[i]}_pb_Ustar.face'
            data = read_facefile(wall_file)
            keys = list(data.keys())
            if "VDF" in config[i]:
                y, x, u_plus, d_plus, u_star, tau, tau_y, tau_x = get_wall_param2D(data[keys[-1]])
                sort_index = np.argsort(x)
                x, tau_x = x[sort_index], tau_x[sort_index]
                tau_x = rho*(-tau_x)
                # save value of u_tau
                u_plus, d_plus, u_star, tau, tau_xprof, tau_yprof = get_values_at_x2D(data[keys[-1]], x_prof)
            else :
                z, y, x, u_plus, d_plus, u_star, tau, tau_x, tau_y, tau_z = get_wall_param3D(data[keys[-1]])
                tau_x = rho*tau_z
                # save value of u_tau
                u_plus, d_plus, u_star, tau, tau_xprof, tau_yprof, tau_zprof = get_values_at_x3D(data[keys[-1]], x_prof)
            u_star_list[i][j] = u_star
            if "VEF" in config[i]:
                y_plus = y_wall_list["VEF"][j]*u_star*rho/mu
            elif "VDF" in config[i]:
                y_plus = y_wall_list["VDF"][j]*u_star*rho/mu
            else:
                y_plus = y_wall_list["PolyMAC"][j]*u_star*rho/mu
            y_plus_cal_list[i][j] = y_plus
            print(f"{config[i]}\t, mesh_{j} :\t u* = {u_star:.4f} m/s,\t\t y+={y_plus:.1f}")
        else:
            # plot tau_x=f(x)
            x, y_demi = abscisses_sonde_yplus(config[i],Ny[j])
            y_plus = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{names[i]}_Y_PLUS.son")[1::,1])
            u_tau, tau_x = get_tau_from_yplus(y_plus, y_demi, rho, mu)
            # save value of u_tau
            u_star = get_value_at_x(u_tau,x,x_prof)
            u_star_list[i][j] = u_star
            y_plus = 2*y_demi*u_star*rho/mu
            y_plus_cal_list[i][j] = y_plus
            print(f"{config[i]}\t, mesh_{j} :\t u* = {u_star:.4f} m/s,\t\t y+={y_plus:.1f}") # 2*y_demi pour etre à la frontière sup de la 1ere maille à la paroi
        if min(tau_x)<min_tau:
            min_tau = min(tau_x)
        if max(tau_x)>max_tau:
            max_tau = max(tau_x)    
        axs[int(i/2),i%2].plot(x, tau_x, label=f"mesh_{j}") # Change plotted variable here (choose u_plus, d_plus, u_star, tau, tau_x or tau_y)
    axs[int(i/2),i%2].plot(x,tau_pred*np.ones(len(x)), "--",label='estimation')
    axs[int(i/2),i%2].legend(fontsize=12)
    axs[int(i/2),i%2].set_xlabel("x [m]", fontsize=10)
    axs[int(i/2),i%2].set_ylabel("tau_x [Pa]", fontsize=10)
    axs[int(i/2),i%2].set_title(f'{config[i]}', fontsize=12)
# Defining custom 'xlim' and 'ylim' values.
custom_xlim = (0, 100)
custom_ylim = (min_tau-0.1, max_tau+0.1)

# Setting the values for all axes.
plt.setp(axs, xlim=custom_xlim, ylim=custom_ylim)

fig.suptitle(f"Contraintes pariétales à t={tmax}s")
fig.tight_layout()
print(f"\nComparaison avec l'estimation : u* = {u_tau_pred:.4f} m/s")
for i in range(nb_mesh):
    print(f" - mesh_{i} y+ à la paroi : y+_VEF = {y_plus_list['VEF'][i]:.1f}\t y+_VDF = {y_plus_list['VDF'][i]:.1f}\t y+_PolyMAC = {y_plus_list['PolyMAC'][i]:.1f}")

### Profils de vitesse
#### Etude de la convergence en maillage

In [ ]:
# # Plot u⁺=f(y⁺)
fig = plt.figure(figsize = (15,4*ceil(nb_method/2)))

axs = fig.subplots(ceil(nb_method/2),2)

for i in range(nb_method):
    file = names[i]
    for j in range(nb_mesh):
        utau = u_star_list[i][j] 
        uplus = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son")[3::3,1])/utau
        if "VDF" in config[i]: # 2D simulation with axisymetric BC
            uplus = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son")[2::2,1])/utau
        yplus = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])*utau/(mu/rho)
        if "VEF" in config[i]: # different mesh (N_vef)
            yplus = np.linspace(y_min_profvef[j], y_max_profvef[j], N_vef[j])*utau/(mu/rho)
            uplus = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE_GRAV.son")[3::3,1])/utau
        axs[int(i/2),i%2].plot(yplus, uplus, "-o",markersize=3,label=f"mesh_{j}: u⁺=f(y⁺)")
        axs[int(i/2),i%2].plot(y_plus_cal_list[i][j]*np.ones(2),np.linspace(4,20,2), linestyle ='dotted', label=f"mesh_{j}: y⁺wall={y_plus_cal_list[i][j]:.0f}")
    axs[int(i/2),i%2].plot(yplus[yplus>10], (1/kappa)*np.log(yplus[yplus>10])+B, "--", label="log law", color="black")
    axs[int(i/2),i%2].legend(fontsize=10)
    axs[int(i/2),i%2].set_xlabel("y+", fontsize=12)
    axs[int(i/2),i%2].set_ylabel("u+", fontsize=12)
    axs[int(i/2),i%2].set_xscale('log')
    axs[int(i/2),i%2].set_title(f'{config[i]}', fontsize=12)
fig.suptitle(f"Profils de vitesse à x={x_prof}m : convergence en maillage")
fig.tight_layout()

In [ ]:
# # Plot u=f(y)

Graph=plot.Graph("Velocity profiles : convergence en maillage",nX=ceil(nb_method/2),nY=2,size=[8,6])
for i in range(nb_method):
    file = names[i]
    Graph.addPlot((int(i/2),i%2),f"{config[i]}")
    comp = 2
    if "VDF" in config[i]: # maillage 2D
        comp=1
    for j in range(nb_mesh):
        if "VEF" in config[i]: # vitesse au centre de gravité pour vef (éviter l'interpolation)
            Graph.addSegment(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE_GRAV.son", linestyle="solid", marker = 'o', markersize=3,compo=comp,label=f"mesh_{j}")
        else:
            Graph.addSegment(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son", linestyle="solid", marker = 'o', markersize=3,compo=comp,label=f"mesh_{j}")
    Graph.legend()
    Graph.label("y [m]", "u [m/s]")

#### Comparaison des différentes méthodes de calcul

In [ ]:
# # Plot u+=f(y+)

Graph=plot.Graph("Profils de vitesse : comparaison des méthodes", nY=nb_mesh,size=[8,5])
for j in range(nb_mesh):
    Graph.addPlot(j,f"mesh_{j}")
    for i in range(nb_method):
        file = names[i]
        utau = u_star_list[i][j] 
        uplus = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son")[3::3,1])/utau
        if "VDF" in config[i]: # 2D simulation with axisymetric BC
            uplus = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son")[2::2,1])/utau
        yplus = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])*utau/(mu/rho)
        if "VEF" in config[i]: # different mesh (N_vef)
            uplus = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE_GRAV.son")[3::3,1])/utau
            yplus = np.linspace(y_min_profvef[j], y_max_profvef[j], N_vef[j])*utau/(mu/rho)
        Graph.add(yplus, uplus, linestyle="solid", marker = 'o', markersize=3,label=f"{config[i]}")
    Graph.add(yplus[yplus>10], (1/kappa)*np.log(yplus[yplus>10])+B, "--", linewidth=2, label="log law", color="black")
    Graph.scale(xscale='log')
    Graph.legend()
    Graph.label("y⁺", "u⁺")
#print("                    Velocity profiles : comparaison des méthodes")

In [ ]:
Graph=plot.Graph("Profils de vitesse : comparaison des méthodes", nY=nb_mesh,size=[8,5])
for j in range(nb_mesh):
    Graph.addPlot(j,f"mesh_{j}")
    for i in range(nb_method):
        file = names[i] 
        u = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son")[3::3,1])
        if "VDF" in config[i]: # 2D simulation with axisymetric BC
            u = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE.son")[2::2,1])
        y = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])
        if "VEF" in config[i]: # different mesh (N_vef)
            u = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_VITESSE_GRAV.son")[3::3,1])
            y = np.linspace(y_min_profvef[j], y_max_profvef[j], N_vef[j])
        Graph.add(y, u, linestyle="solid", marker = 'o', markersize=3,label=f"{config[i]}")
    Graph.legend()
    Graph.label("y [m]", "u [m/s]")

### Traceurs de la turbulence
#### Etude de la convergence en maillage

In [ ]:
fig = plt.figure(figsize = (15,5*nb_method))

axs = fig.subplots(nb_method,2)
for i in range(nb_method):
    file = names[i]
    for j in range(nb_mesh):
        k = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_K.son")[1::,1])
        y = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])
        if "VEF" in config[i]: # different mesh (N_vef)
            y = np.linspace(y_min_profvef[j], y_max_profvef[j], N_vef[j])
        axs[i,0].plot(y,k, linestyle="solid", marker = 'o', markersize=3,label=f"mesh_{j}")
    axs[i,0].legend()
    axs[i,0].set_xlabel("y [m]")
    axs[i,0].set_ylabel("k [m²/s²]")
    axs[i,0].set_title(f'{config[i]}', fontsize=12)


for i in range(nb_method):
    file = names[i]
    for j in range(nb_mesh):
        diss = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_DISS.son")[1::,1])
        k = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_K.son")[1::,1])
        if "tau" in config[i]:
            epsilon = 0.09*k/diss
        elif "omega" in config[i]:
            epsilon = 0.09*k*diss
        else:
            epsilon = diss
        y = np.linspace(y_min_prof[j], y_max_prof[j], nb_points_prof[j])
        if "VEF" in config[i]: # different mesh (N_vef)
            y = np.linspace(y_min_profvef[j], y_max_profvef[j], N_vef[j])
        axs[i,1].plot(y,epsilon, "-o", markersize=3,label=f"mesh_{j}")
    axs[i,1].legend()
    axs[i,1].set_xlabel("y [m]", fontsize=12)
    axs[i,1].set_ylabel("epsilon [m²/s³]", fontsize=12)
    axs[i,1].set_title(f'{config[i]}', fontsize=12)
fig.suptitle(f"Traceurs de turbulence : convergence en maillage")
fig.tight_layout()

#### Comparaison des différentes méthodes de calcul

In [ ]:
Graph=plot.Graph("Traceurs de la turbulence : comparaison des méthodes",nX=2, nY=nb_mesh,size=[8,6])
for j in range(nb_mesh):
    if nb_mesh==1:
        indice = 0
    else:
        indice = (0,j)
    Graph.addPlot(indice,f"mesh_{j}")
    for i in range(nb_method):
        file = names[i]
        k = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_K.son")[1::,1])
        y = np.linspace(y_max_prof[j], y_min_prof[j], nb_points_prof[j])
        if "VEF" in config[i]: # different mesh (N_vef)
            y = np.linspace(y_max_profvef[j], y_min_profvef[j], N_vef[j])
        Graph.add(y,k, "-o", markersize=3,label=f"{config[i]}")
        #Graph.addSegment(build+f"/{config[i]}/mesh_{j}/{file}_K.son", linestyle="solid", marker = 'o', markersize=3,label=f"{config[i]}")
    Graph.legend()
    Graph.label("y [m]", "k [m²/s²]")
    
for j in range(nb_mesh):
    if nb_mesh==1:
        indice = 1
    else:
        indice = (1,j)
    Graph.addPlot(indice,f"mesh_{j}")
    for i in range(nb_method):
        file = names[i]
        diss = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_DISS.son")[1::,1])
        k = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_K.son")[1::,1])
        if "tau" in config[i]:
            epsilon = 0.09*k/diss
        elif "omega" in config[i]:
            epsilon = 0.09*k*diss
        else:
            epsilon = diss
        y = np.linspace(y_max_prof[j], y_min_prof[j], nb_points_prof[j])
        if "VEF" in config[i]: # different mesh (N_vef)
            y = np.linspace(y_max_profvef[j], y_min_profvef[j], N_vef[j])
        Graph.add(y,epsilon, "-o", markersize=3,label=f"{config[i]}")
    Graph.legend()
    Graph.label("y [m]", "epsilon [m²/s³]")

### Perte de charge

In [ ]:
f_guess = 0.02
f = facteur_frottement(Re_Dh,f_guess) # coefficient de frottement donnée par la formule de Von Karman et Nikurade (Re>1e5)
v_m = inlet_velocity # vitesse moyenne sur une section
L = 100-50 # [m] : longueur du conduit
delta_p_estime = -f*L/Dh*rho*v_m**2/2
print(f"Estimation de la perte de charge avec la formule de Von Karman et Nikuradse (comme le diagramme de Moody):\nDeltaP = {delta_p_estime:.1f} Pa")

In [ ]:
for j in range(nb_mesh):
    for i in range(nb_method):
        file = names[i]
        #p = np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_PRESSION_IO.son")[1:3,1])
        p_inlet = np.mean(np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_PRESSION_I.son")[1::,1]))
        p_outlet = np.mean(np.array(plot.loadText(build+f"/{config[i]}/mesh_{j}/{file}_PRESSION_O.son")[1::,1]))
        delta_p = p_outlet-p_inlet
        if "k-epsilon" in config[i]:
            delta_p = rho*delta_p
        #delta_u2 = u[1]**2-u[0]**2
        print(f"{config[i]}\t-\tmesh_{j}\t: deltaP = {delta_p:.1f} Pa\tEcart relatif : {100*(abs(delta_p)-abs(delta_p_estime))/abs(delta_p_estime):.0f}%")


In [ ]:
#fig=visit.Show(build+f"/VDF_k-epsilon/mesh_0/{file_pb_hydr}.lata","Pseudocolor","VITESSE_X_ELEM_dom")

# to add a list of viewing options
#fig.visuOptions(['no_databaseinfo','no_legend'])

# To display the visu with the initialization visit.Show()
#fig.plot()